In [51]:
import pandas as pd

# ========== 1. 读取数据 ==========
df = pd.read_csv("data/Camden_data.csv")

# ========== 2. 数据筛选 ==========
filtered_df = df[
    (df['property_type'].str.lower() == 'flat') &
    (df['ward22nm'] == 'Bloomsbury') &
    (df['built_form'].str.lower().str.contains('-terrace')) &
    (df['wall_type'].str.lower() != 'other')
].copy()

In [52]:
print(filtered_df.head())

         uprn  os_topo_toid   easting  northing postcode_locator  \
6413  5006230  1.000000e+12  530203.0  181732.0         WC1B 5BH   
6414  5171576  1.000000e+12  530309.0  181747.0         WC1A 2NS   
6417  5136568  1.000000e+12  530334.0  181768.0         WC1A 2QN   
6418  5006231  1.000000e+12  530203.0  181732.0         WC1B 5BH   
6423  5171575  1.000000e+12  530309.0  181747.0         WC1A 2NS   

     administrative_area     oa21cd   lsoa21cd     lsoa21nm   lsoa11cd  \
6413              Camden  E00004169  E01000855  Camden 028A  E01000855   
6414              Camden  E00004169  E01000855  Camden 028A  E01000855   
6417              Camden  E00004169  E01000855  Camden 028A  E01000855   
6418              Camden  E00004169  E01000855  Camden 028A  E01000855   
6423              Camden  E00004169  E01000855  Camden 028A  E01000855   

         lsoa11nm   ward22cd    ward22nm property_type   built_form  \
6413  Camden 028A  E05013653  Bloomsbury          flat  mid-terrace   
6414

In [54]:
import pandas as pd
import numpy as np

# ===== 1. 精选字段 =====
columns_to_keep = [
    'construction_age_band', 'wall_type', 'wall_insulation', 'roof_type', 'roof_insulation',
    'floor_type', 'glazing_type', 'built_form', 'property_type',
    'total_floor_area', 'estimated_floor_count', 'number_habitable_rooms',
    'epc_score', 'main_heat_type', 'main_fuel_type', 'energy_consumption', 'tenure'
]
columns_to_keep = [c for c in columns_to_keep if c in filtered_df.columns]
filtered_df = filtered_df[columns_to_keep].copy()

# ===== 2. 年代标准化完全覆盖8个类型 =====
def standardize_age_band(age_band):
    ab = str(age_band).strip().lower()
    if ab in ['pre-1900', '1900-1929']:
        return 'pre-1920'
    elif ab == '1930-1949':
        return '1930-1949'
    elif ab == '1950-1966':
        return '1950-1966'
    elif ab == '1967-1982':
        return '1967-1982'
    elif ab == '1983-1995':
        return '1983-1995'
    elif ab == '1996-2011':
        return '1996-2011'
    elif ab == '2012-onwards':
        return '2012-onwards'
    else:
        return np.nan

filtered_df['lookup_age_band'] = filtered_df['construction_age_band'].apply(standardize_age_band)

# ===== 3. 全查找相关字段小写去空格 =====
for col in [
    'wall_type', 'wall_insulation', 'roof_type', 'roof_insulation', 'floor_type', 'glazing_type'
]:
    if col in filtered_df.columns:
        filtered_df[col] = filtered_df[col].astype(str).str.strip().str.lower()

# ===== 4. 构建查找表（只保留墙和屋顶/窗体） =====
age_band_list = ['pre-1920', '1930-1949', '1950-1966', '1967-1982', '1983-1995', '1996-2011', '2012-onwards']

# -- 墙体
wall_lookup = pd.DataFrame({
    'lookup_age_band': age_band_list*4,
    'wall_type': ['solid']*7 + ['solid']*7 + ['cavity']*7 + ['cavity']*7,
    'wall_insulation': ['uninsulated']*7 + ['insulated']*7 + ['uninsulated']*7 + ['insulated']*7,
    'u_value_wall': [
        1.7,1.7,1.36,1.35,1.0,0.37,0.16,    # solid/unins
        0.55,0.55,0.55,0.35,0.35,0.19,0.16, # solid/ins
        1.5,1.5,1.0,1.0,0.7,0.28,0.16,      # cavity/unins
        0.37,0.37,0.32,0.32,0.25,0.18,0.16  # cavity/ins
    ]
})

# -- 屋顶
roof_comb = []
roof_uv = []
for ab in age_band_list:
    # pitched-unins+ins
    roof_comb += [(ab, 'pitched', 'uninsulated'), (ab, 'pitched', 'insulated')]
    roof_uv += [2.3, 0.25]
    # flat-unins+ins
    roof_comb += [(ab, 'flat', 'uninsulated'), (ab, 'flat', 'insulated')]
    roof_uv += [2.3, 0.28]
    # room in roof
    roof_comb += [(ab, 'room in roof', 'room in roof')]
    roof_uv += [0.35]
    # another dwelling above
    roof_comb += [(ab, 'another dwelling above', 'another dwelling above')]
    roof_uv += [0.0]
roof_lookup = pd.DataFrame(roof_comb, columns=['lookup_age_band','roof_type','roof_insulation'])
roof_lookup['u_value_roof'] = roof_uv

# -- 窗
glazing_lookup = pd.DataFrame({
    'glazing_type': ['single/partial','double/triple','secondary'],
    'u_value_glazing': [5.75,2.4,2.82]
})

# ===== 5. 查表函数（用字符串） =====
def get_wall_u(row):
    try:
        match = wall_lookup[
            (wall_lookup['lookup_age_band'] == row['lookup_age_band']) &
            (wall_lookup['wall_type'] == row['wall_type']) &
            (wall_lookup['wall_insulation'] == row['wall_insulation'])
        ]
        return match['u_value_wall'].iloc[0] if not match.empty else np.nan
    except: return np.nan

def get_roof_u(row):
    try:
        match = roof_lookup[
            (roof_lookup['lookup_age_band'] == row['lookup_age_band']) &
            (roof_lookup['roof_type'] == row['roof_type']) &
            (roof_lookup['roof_insulation'] == row['roof_insulation'])
        ]
        return match['u_value_roof'].iloc[0] if not match.empty else np.nan
    except: return np.nan

def get_glazing_u(row):
    try:
        match = glazing_lookup[
            (glazing_lookup['glazing_type'] == row['glazing_type'])
        ]
        return match['u_value_glazing'].iloc[0] if not match.empty else np.nan
    except: return np.nan

filtered_df['u_value_wall'] = filtered_df.apply(get_wall_u, axis=1)
filtered_df['u_value_roof'] = filtered_df.apply(get_roof_u, axis=1)
filtered_df['u_value_floor'] = 0.25  # 直接赋值为0.25，不做查表
filtered_df['u_value_glazing'] = filtered_df.apply(get_glazing_u, axis=1)

# ===== 7. 输出结果 =====
filtered_df.to_csv('cleaned_flat_terraced_data_final.csv',index=False)
print(filtered_df.head())
print(filtered_df.info())

u_value_roof nan EXAMPLES:
      construction_age_band     roof_type roof_insulation lookup_age_band
6656               pre-1900  room in roof       insulated        pre-1920
6690               pre-1900  room in roof     uninsulated        pre-1920
8207              1900-1929  room in roof     uninsulated        pre-1920
9262              1900-1929  room in roof       insulated        pre-1920
11072             1996-2011  room in roof       insulated       1996-2011
     construction_age_band wall_type wall_insulation               roof_type  \
6413              pre-1900     solid     uninsulated  another dwelling above   
6414              pre-1900     solid     uninsulated                    flat   
6417              pre-1900     solid     uninsulated  another dwelling above   
6418             1900-1929     solid     uninsulated  another dwelling above   
6423              pre-1900     solid     uninsulated  another dwelling above   

             roof_insulation    glazing_type   b

In [56]:
core_cols = [
    'construction_age_band', 'wall_type', 'wall_insulation', 'roof_type', 'roof_insulation',
    'glazing_type', 'built_form',
    'total_floor_area', 'estimated_floor_count', 'number_habitable_rooms',
    'epc_score', 'main_heat_type', 'main_fuel_type', 'energy_consumption',
    'lookup_age_band', 'u_value_wall', 'u_value_roof', 'u_value_floor', 'u_value_glazing'
]
filtered_df_clean = filtered_df[core_cols].dropna().reset_index(drop=True)

In [57]:
import math

delta_T = 18  # 英国室内外年均温差（单位：摄氏度，默认值，必要可自行微调）

def calc_areas(row):
    tfa = row['total_floor_area']
    # built_form 用于细分风格，这里设end-terrace/mid-terrace不影响公式
    # 屋顶面积
    if row['roof_type'] == 'pitched':
        roof_area = tfa / math.cos(math.radians(30))
    elif row['roof_type'] == 'flat':
        roof_area = tfa
    else:  # another dwelling above、room in roof 等
        roof_area = 0
    # 墙体面积
    wall_area = tfa * 2.1
    # 地面面积
    floor_area = tfa
    # 窗面积
    glazing_area = tfa * 0.18
    return pd.Series([wall_area, roof_area, floor_area, glazing_area])

filtered_df_clean[['wall_area', 'roof_area', 'floor_area', 'glazing_area']] = \
    filtered_df_clean.apply(calc_areas, axis=1)

# 白盒Q_total计算
filtered_df_clean['Q_total'] = (
    filtered_df_clean['u_value_wall'] * filtered_df_clean['wall_area'] +
    filtered_df_clean['u_value_roof'] * filtered_df_clean['roof_area'] +
    filtered_df_clean['u_value_floor'] * filtered_df_clean['floor_area'] +
    filtered_df_clean['u_value_glazing'] * filtered_df_clean['glazing_area']
) * delta_T

In [58]:
cat_cols = [
    'built_form', 'wall_type', 'wall_insulation', 'roof_type', 'roof_insulation',
    'glazing_type', 'main_heat_type', 'main_fuel_type', 'lookup_age_band'
]
num_cols = [
    'epc_score', 'estimated_floor_count',
    'wall_area', 'roof_area', 'floor_area', 'glazing_area',
    'u_value_wall', 'u_value_roof', 'u_value_floor', 'u_value_glazing',
    'Q_total'
]
X = pd.get_dummies(filtered_df_clean[cat_cols + num_cols], drop_first=True)
y = filtered_df_clean['energy_consumption']  # 预测目标

# 检查下X和y无缺失
X = X.fillna(0)
y = y.fillna(0)

# ML Model

In [63]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 你的数据 X, y，已经完成特征工程
from sklearn.model_selection import train_test_split

# 切分数据
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.2)

# 待测试的模型列表（可根据实际增减）
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'DecisionTree': DecisionTreeRegressor(random_state=100, max_depth=10),
    'RandomForest': RandomForestRegressor(random_state=100, n_estimators=100, max_depth=7),
    'GradientBoosting': GradientBoostingRegressor(random_state=100, n_estimators=100, max_depth=5),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5)
}

# 记录性能
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    results.append({
        'Model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2
    })

# 输出对比表
results_df = pd.DataFrame(results).sort_values('R2', ascending=False)
print(results_df)

# 如用notebook，可以直接展示为表格：
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,3))
sns.barplot(x='R2', y='Model', data=results_df, palette="viridis")
plt.title("模型R2对比")
plt.show()

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

              Model          MAE         RMSE        R2
0  LinearRegression  1513.445828  3942.459028  0.867881
1             Ridge  1510.798369  3946.637948  0.867601
4  GradientBoosting   905.753751  4976.023847  0.789527
3      RandomForest  1179.286088  4980.588437  0.789141
5        KNeighbors  1426.205069  5512.601997  0.741688
2      DecisionTree  1181.199634  5559.929994  0.737233


/tmp/ipykernel_896/2929478629.py:45: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='R2', y='Model', data=results_df, palette="viridis")


In [60]:
import joblib
joblib.dump(rf,     'rf_model.joblib')
joblib.dump(X.columns, 'rf_features.joblib')

['rf_features.joblib']

2025-06-14 19:46:21.143 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.192 
  command:

    streamlit run /opt/conda/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-14 19:46:21.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 19:46:21.195 Thread 'MainThread': missi